In [58]:
from os.path import expanduser, join
from pandas import read_csv

electricity_consumption_by_year_table_path = join(
    '..',
    'datasets',
    'oregon-electricity-consumption-by-year-for-sixteen-cities.csv')
electricity_consumption_by_year_table = read_csv(
    electricity_consumption_by_year_table_path)
electricity_consumption_by_year_table[:11]

,Entity,State,Ownership,Customers (Count),Sales (Megawatthours),Revenues (Thousands Dollars),Average Price (cents/kWh),Year
0,City of Ashland,OR,Public,10422,10942,169437,6.457857,2004
1,City of Ashland,OR,Public,11310,12247,169156,7.240062,2005
2,City of Ashland,OR,Public,11503,13874,173811,7.982234,2006
3,City of Ashland,OR,Public,10730,12518,175565,7.130000,2007
4,City of Ashland,OR,Public,11879,180514,12989,7.200000,2008
5,City of Ashland,OR,Public,11549,165688,11755,7.090000,2009
6,City of Ashland,OR,Public,11056,173022,11306,6.530000,2010
7,City of Ashland,OR,Public,11390,175389,12436,7.090000,2011
8,City of Ashland,OR,Municipal,11688,172148,12475,7.246671,2012
9,City of Ashland,OR,Municipal,11117,176839,13313,7.528317,2013


In [59]:
name_column = electricity_consumption_by_year_table_name_column = 'Entity'
consumption_column = electricity_consumption_by_year_table_consumption_column = 'Sales (Megawatthours)'

In [60]:
for name, local_consumption_by_year_table in electricity_consumption_by_year_table.groupby(
        electricity_consumption_by_year_table_name_column):
    print(name)
    t = local_consumption_by_year_table
    break

City of Ashland


In [61]:
def define_get_consumption_in_kilowatt_hours(consumption_column):
    x = consumption_column.lower()
    
    if 'kwh' in x or ('kilowatt' in x and 'hour' in x):
        conversion_factor = 10 ** 0
    if 'mwh' in x or ('megawatt' in x and 'hour' in x):
        conversion_factor = 10 ** 3
    if 'gwh' in x or ('gigawatt' in x and 'hour' in x):
        conversion_factor = 10 ** 6
    
    return lambda x: x * conversion_factor

get_consumption_in_kilowatt_hours = define_get_consumption_in_kilowatt_hours(
    consumption_column)
maximum_consumption_in_kilowatt_hours_per_year = get_consumption_in_kilowatt_hours(
    t[consumption_column].max())
maximum_consumption_in_kilowatt_hours_per_year

180514000

In [62]:
# Get peak_demand

def estimate_peak_demand_in_kilowatts(
        maximum_consumption_in_kilowatt_hours_per_year,
        peak_hours_per_year,
        peak_demand_in_kilowatts_as_fraction_of_estimated_consumption_in_kilowatt_hours_per_peak_hour):    
    """
    Estimate peak demand in order to size system capacity.
    The main assumption is that most electricity consumption happens during peak hours.
    Then the maximum consumption per year divided by the peak hours per year
    is approximately the amount of electricity consumed per peak hour.
    """
    estimated_consumption_in_kilowatt_hours_per_peak_hour = maximum_consumption_in_kilowatt_hours_per_year / float(peak_hours_per_year)
    return peak_demand_in_kilowatts_as_fraction_of_estimated_consumption_in_kilowatt_hours_per_peak_hour * estimated_consumption_in_kilowatt_hours_per_peak_hour

peak_hours_per_year = 1460
peak_demand_in_kilowatts_as_fraction_of_estimated_consumption_in_kilowatt_hours_per_peak_hour = 0.4
peak_demand_in_kilowatts = estimate_peak_demand_in_kilowatts(
    maximum_consumption_in_kilowatt_hours_per_year,
    peak_hours_per_year,
    peak_demand_in_kilowatts_as_fraction_of_estimated_consumption_in_kilowatt_hours_per_peak_hour)
peak_demand_in_kilowatts

49455.890410958906

In [63]:
# Get desired system capacity that will serve the entire community

distribution_loss_percent = 15
desired_system_capacity_in_kilowatts = peak_demand_in_kilowatts / float(
    1 - distribution_loss_percent / 100.)
desired_system_capacity_in_kilowatts

58183.400483481069

In [64]:
from os.path import expanduser, join
from pandas import read_csv

transformer_table_path = join(
    '..',
    'datasets',
    'example-transformer-by-capacity.csv')
transformer_table = read_csv(transformer_table_path)
transformer_table

,Capacity in Kilowatts,Installation Worker Salary in Dollars per Year,Installation Worker Count,Installation Time in Years,Installation Material Cost in Dollars,Maintenance Worker Salary in Dollars per Year,Maintenance Worker Count,Maintenance Time in Years,Maintenance Material Cost in Dollars per Year,Replacement Worker Salary in Dollars per Year,Replacement Worker Count,Replacement Time in Years,Replacement Material Cost in Dollars
0,1000,50000,2,2,5000,60000,2,5,500,50000,3,3,7500
1,500,20000,1,1,1000,30000,1,3,100,20000,1,1,1500


In [65]:
capacity_column = 'Capacity in Kilowatts'

In [66]:
# Choose transformer type
eligible_transformer_table = transformer_table[
    transformer_table[capacity_column] < desired_system_capacity_in_kilowatts]
eligible_transformer_table

,Capacity in Kilowatts,Installation Worker Salary in Dollars per Year,Installation Worker Count,Installation Time in Years,Installation Material Cost in Dollars,Maintenance Worker Salary in Dollars per Year,Maintenance Worker Count,Maintenance Time in Years,Maintenance Material Cost in Dollars per Year,Replacement Worker Salary in Dollars per Year,Replacement Worker Count,Replacement Time in Years,Replacement Material Cost in Dollars
0,1000,50000,2,2,5000,60000,2,5,500,50000,3,3,7500
1,500,20000,1,1,1000,30000,1,3,100,20000,1,1,1500


In [67]:
# Choose largest capacity from available system capacities that is less than the desired capacity
if len(eligible_transformer_table):
    selected_transformer_table = eligible_transformer_table[
        eligible_transformer_table[
            capacity_column
        ] == eligible_transformer_table[
            capacity_column
        ].max()]
else:
    selected_transformer_table = transformer_table[
        transformer_table[
            capacity_column
        ] == transformer_table[
            capacity_column
        ].min()]    
selected_transformer = selected_transformer_table.ix[selected_transformer_table.index[0]]
selected_transformer

Capacity in Kilowatts                              1000
Installation Worker Salary in Dollars per Year    50000
Installation Worker Count                             2
Installation Time in Years                            2
Installation Material Cost in Dollars              5000
Maintenance Worker Salary in Dollars per Year     60000
Maintenance Worker Count                              2
Maintenance Time in Years                             5
Maintenance Material Cost in Dollars per Year       500
Replacement Worker Salary in Dollars per Year     50000
Replacement Worker Count                              3
Replacement Time in Years                             3
Replacement Material Cost in Dollars               7500
Name: 0, dtype: int64

In [68]:
selected_transformer_capacity_in_kilowatts = selected_transformer_table[capacity_column].values[0]
selected_transformer_capacity_in_kilowatts

1000

In [69]:
# Get selected_transformer_count
from math import ceil
selected_transformer_count = int(ceil(
    desired_system_capacity_in_kilowatts / float(selected_transformer_capacity_in_kilowatts)))
selected_transformer_count

59

In [70]:
[
    capacity_column,
    installation_worker_salary_column,
    installation_worker_count_column,
    installation_time_column,
    installation_material_cost_column,
    maintenance_worker_salary_column,
    maintenance_worker_count_column,
    maintenance_time_column,
    maintenance_material_cost_column,
    replacement_worker_salary_column,
    replacement_worker_count_column,
    replacement_time_column,
    replacement_material_cost_column,
] = [
    'Capacity in Kilowatts',

    'Installation Worker Salary in Dollars per Year',
    'Installation Worker Count',
    'Installation Time in Years',
    'Installation Material Cost in Dollars',

    'Maintenance Worker Salary in Dollars per Year',
    'Maintenance Worker Count',
    'Maintenance Time in Years',
    'Maintenance Material Cost in Dollars per Year',

    'Replacement Worker Salary in Dollars per Year',
    'Replacement Worker Count',
    'Replacement Time in Years',
    'Replacement Material Cost in Dollars',        
]

In [71]:
def get_labor_material_time(
        selected_transformer,
        selected_transformer_count,
        worker_salary_column,
        worker_count_column,
        material_cost_column,
        time_column):
    d, k = selected_transformer, selected_transformer_count
    worker_salary = d[worker_salary_column]
    worker_count = d[worker_count_column] * k
    labor_cost_per_year = worker_salary * worker_count
    material_cost_per_year = d[material_cost_column] * k
    year_count = d[time_column]
    return [labor_cost_per_year, material_cost_per_year, year_count]

get_labor_material_time(
    selected_transformer,
    selected_transformer_count,
    installation_worker_salary_column,
    installation_worker_count_column,
    installation_material_cost_column,
    installation_time_column)

[5900000, 295000, 2]

In [72]:
# Get cost breakdown
[
    installation_labor_cost_in_dollars_per_year,
    installation_material_cost_in_dollars,
    installation_time_in_years,
] = get_labor_material_time(
    selected_transformer,
    selected_transformer_count,
    installation_worker_salary_column,
    installation_worker_count_column,
    installation_material_cost_column,
    installation_time_column)

In [73]:
[
    maintenance_labor_cost_in_dollars_per_year,
    maintenance_material_cost_in_dollars_per_year,
    maintenance_time_in_years,
] = get_labor_material_time(
    selected_transformer,
    selected_transformer_count,
    maintenance_worker_salary_column,
    maintenance_worker_count_column,
    maintenance_material_cost_column,
    maintenance_time_column)

In [74]:
[
    replacement_labor_cost_in_dollars_per_year,
    replacement_material_cost_in_dollars,
    replacement_time_in_years,
] = get_labor_material_time(
    selected_transformer,
    selected_transformer_count,
    replacement_worker_salary_column,
    replacement_worker_count_column,
    replacement_material_cost_column,
    replacement_time_column)

In [75]:
year_count = len(t)
year_count

11

In [76]:
import numpy as np

def get_cost_breakdown_table(
        prefix,
        year_count,
        installation_labor_cost_in_dollars_per_year,
        installation_material_cost_in_dollars,
        installation_time_in_years,
        maintenance_labor_cost_in_dollars_per_year,
        maintenance_material_cost_in_dollars_per_year,
        maintenance_time_in_years,
        replacement_labor_cost_in_dollars_per_year,
        replacement_material_cost_in_dollars,
        replacement_time_in_years):

    installation_labor_costs = np.zeros(year_count)
    installation_labor_costs[:installation_time_in_years] = installation_labor_cost_in_dollars_per_year

    installation_material_costs = np.zeros(year_count)
    installation_material_costs[0] = installation_material_cost_in_dollars

    maintenance_labor_costs = np.zeros(year_count)
    maintenance_labor_costs[installation_time_in_years:] = maintenance_labor_cost_in_dollars_per_year

    maintenance_material_costs = np.zeros(year_count)
    maintenance_material_costs[installation_time_in_years:] = maintenance_material_cost_in_dollars_per_year

    replacement_labor_costs = np.zeros(year_count)
    index1 = installation_time_in_years + maintenance_time_in_years
    index2 = index1 + replacement_time_in_years
    while index1 < year_count:
        replacement_labor_costs[index1:index2] = replacement_labor_cost_in_dollars_per_year
        index1 = index2 + maintenance_time_in_years
        index2 = index1 + replacement_time_in_years

    replacement_material_costs = np.zeros(year_count)
    index1 = installation_time_in_years + maintenance_time_in_years
    while index1 < year_count:
        replacement_material_costs[index1] = replacement_material_cost_in_dollars
        index1 += maintenance_time_in_years + replacement_time_in_years
    replacement_material_costs

    return DataFrame({
        prefix + ' Installation Labor Cost ($)': installation_labor_costs,
        prefix + ' Installation Material Cost ($)': installation_material_costs,
        prefix + ' Maintenance Labor Cost ($)': maintenance_labor_costs,
        prefix + ' Maintenance Material Cost ($)': maintenance_material_costs,
        prefix + ' Replacement Labor Cost ($)': replacement_labor_costs,
        prefix + ' Replacement Material Cost ($)': replacement_material_costs,
    })

get_cost_breakdown_table(
    'LV Transformer',
    year_count,
    installation_labor_cost_in_dollars_per_year,
    installation_material_cost_in_dollars,
    installation_time_in_years,
    maintenance_labor_cost_in_dollars_per_year,
    maintenance_material_cost_in_dollars_per_year,
    maintenance_time_in_years,
    replacement_labor_cost_in_dollars_per_year,
    replacement_material_cost_in_dollars,
    replacement_time_in_years)

,LV Transformer Installation Labor Cost ($),LV Transformer Installation Material Cost ($),LV Transformer Maintenance Labor Cost ($),LV Transformer Maintenance Material Cost ($),LV Transformer Replacement Labor Cost ($),LV Transformer Replacement Material Cost ($)
0,5900000,295000,0,0,0,0
1,5900000,0,0,0,0,0
2,0,0,7080000,29500,0,0
3,0,0,7080000,29500,0,0
4,0,0,7080000,29500,0,0
5,0,0,7080000,29500,0,0
6,0,0,7080000,29500,0,0
7,0,0,7080000,29500,8850000,442500
8,0,0,7080000,29500,8850000,0
9,0,0,7080000,29500,8850000,0


In [77]:
# For each location,

    # + Get peak_demand
    # + Get desired system capacity in kilowatts
    
    # + Load transformer table
    # + Choose transformer type
    # + Get selected_transformer_count
    
    # Get cost breakdown
    # Get discounted cost
    # Get levelized cost

# Get aggregated cost breakdown
# Get aggregated discounted cost
# Get aggregated levelized cost

In [ ]:
def prepare_pre_network_grid_costs(
        consumption_by_year_table,
        consumption_column):
    d = {}
    return d

In [ ]:
# For each location,
for name, local_consumption_by_year_table in electricity_consumption_by_year_table.groupby(
        electricity_consumption_by_year_table_name_column):
    prepare_pre_network_grid_costs(
        local_consumption_by_year_table, electricity_consumption_by_year_table_consumption_column)